In [1]:
import scipy
import numpy as np
import json

import spacy
import tensorflow
import keras
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, KeywordsOptions, EntitiesOptions, SemanticRolesOptions

import nltk
from nltk.stem import WordNetLemmatizer

# from __future__ import absolute_import
# from __future__ import division, print_function, unicode_literals

# from sumy.parsers.html import HtmlParser
# from sumy.parsers.plaintext import PlaintextParser
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lsa import LsaSummarizer as Summarizer
# from sumy.nlp.stemmers import Stemmer
# from sumy.utils import get_stop_words

import inflection
import en_core_web_lg
import language_check
nlp = spacy.load('en_core_web_lg')

#nltk.download('punkt')
#nltk.download('wordnet')

Using TensorFlow backend.


In [2]:
#Load credentials from file (Store credentials in json format)
with open('credentials.json') as f:
    data = json.load(f)
url = data["url"]
username = data["username"]
password = data["password"]

In [3]:
# set grammar checker
grammar_tool = language_check.LanguageTool('en-US')

In [4]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
  username=username,
  password=password,
  version='2018-03-16')

response = natural_language_understanding.analyze(
    url='https://www.nytimes.com/2018/07/16/opinion/trump-and-putin-vs-america.html?action=click&pgtype=Homepage&clickSource=story-heading&module=opinion-c-col-left-region&region=opinion-c-col-left-region&WT.nav=opinion-c-col-left-region',
    language='en',
    features=Features(
        keywords=KeywordsOptions(
            sentiment=False,
            emotion=False,
            limit=20),
    entities=EntitiesOptions(
        sentiment=False,
            emotion=False,
            limit=50),
    semantic_roles=SemanticRolesOptions()
  ))
entities = response['entities']
keywords = response['keywords']
semantic = response['semantic_roles']

In [5]:
# Question 1
# Extract keywords and entities
# define type of words
# create questions
def Q1(x):
    return {
        'Person': "Who is ",
        'Location': "Where is "
    }.get(x, "What is ")

with open("Questions1.txt", "w") as file:
    for en in entities:
        text = Q1(en['type']) + en['text'] + "?"
        matches = grammar_tool.check(text)
        correct_text = language_check.correct(text, matches)
        file.write("%s\n" % correct_text)

In [6]:
#Question 2
# Extract keywords and entities
# define plurality of keywords using nltk
# create question What are? Who are? 
# TODO. How to determine plurality of word?
# TODO. If condition does not suffice, what else?


wnl = WordNetLemmatizer()

def isplural(word):
    lemma = wnl.lemmatize(word, 'n')
    plural = True if word is not lemma else False
    return plural

In [7]:
# Question 4
# Why (factual statement)?
# TODO. How to determine which sentences are important enough to negate? TEXTSUM, sumy NOTWORKING

# Question 5
# What if (negated statements)?
# TODO. How to negate sentences? add not, find antonym
# TODO. How to determine which sentences are important enough to negate? summarize? gensim, pyteaser, pytextrank, TEXTSUM, sumy


# LANGUAGE = "english"
# SENTENCES_COUNT = 6
# if __name__ == "__main__":
#     #url = "https://en.wikipedia.org/wiki/Steven_Spielberg"
#     #parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
#     # or for plain text files
#     parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
#     stemmer = Stemmer(LANGUAGE)

#     summarizer = Summarizer(stemmer)
#     summarizer.stop_words = get_stop_words(LANGUAGE)

#     for sentence in summarizer(parser.document, SENTENCES_COUNT):
#         result = natural_language_understanding.analyze(text=str(sentence), features=Features(semantic_roles=SemanticRolesOptions()))
#         #print(json.dumps(result, indent=2))
#         for sentence in result['semantic_roles']:
#             print(len(sentence))
#             if len(sentence) == 4:
#                 print("Why Does", sentence['subject']['text'], verb, obj)
#                 print("What if", sentence['subject']['text'], "did not",verb, obj)



Q4 = []
Q5 = []

listOfPlurals = ["they", "some", "most", "we"]

for sentence in semantic:
    if len(sentence) == 4:
        verb = sentence['action']['normalized']
        subj = sentence['subject']['text'].lower()
        obj = sentence['object']['text']
        if verb is not "s" and verb != "be":
            plurality = subj is not inflection.singularize(subj) or subj in listOfPlurals
            print(verb)
            if plurality:
                Q4.append("Why do " + subj + " " + verb + " " + obj + "?")
            else:
                Q4.append("Why does " + subj + " " + verb + " " + obj + "?")
            Q5.append("What if " + subj + " did not " + verb + " " + obj + "?")
        elif verb == "be":
            plurality = subj is not inflection.singularize(subj) or subj in listOfPlurals
            if plurality:
                Q4.append("Why are " + subj + " " + obj + "?")
                Q5.append("What if " + subj + " were not " + obj + "?")
            else:
                Q4.append("Why is " + subj + " " + obj + "?")
                Q5.append("What if " + subj + " was not " + obj + "?")
        
with open("Questions4.txt", "w", encoding='UTF-8') as file:
    for q in Q4:
        matches = grammar_tool.check(q)
        corrected_q = language_check.correct(q, matches)
        file.write("%s\n" % corrected_q)
with open("Questions5.txt", "w", encoding='UTF-8') as file:
    for q in Q5:
        matches = grammar_tool.check(q)
        corrected_q = language_check.correct(q, matches)
        file.write("%s\n" % corrected_q)

# TODO: Check relevance to text
# TODO: lets do grammar check first and delete or fix the grammar before comparing relevance to text



enjoy
enjoy play
reinforce
have
to make
engage
violate
to ``preserve
vacate
will be
issue
sit
have
see
like
add
will
ll bet
get
be ask
believe
throw
actually say
do
to blame
make
say
keep
dis
blame
try
try to defeat
make
endorse
exceed


In [25]:
# make sure to use larger model!
exceptList = ['DATE', 'MONEY', 'TIME', 'ORDINAL', 'QUANTITY', 'CARDINAL', 'PERCENT']
with open('article.txt', 'r') as file:
    textQ3 = file.readlines()
for line in textQ3:
    doc = nlp(line)
    for ent in doc.ents:
        if ent.label_ not in exceptList:
            print(str(ent.text), ent.label_)


@@11241 ORG
Sol Yurick PERSON
The Warriors WORK_OF_ART
Yurick GPE
Anabasis NORP
Greek NORP
Yurick GPE
Bronx GPE
Cyrus PERSON
Greeks NORP
Anabasis NORP
Walter Hill PERSON
Warriors PRODUCT
Bronx GPE
Coney Island LOC
Yurick GPE
Baseball Furies WORK_OF_ART
Riverside Park LOC
Manhattan GPE
Coney Island LOC
Cyrus PERSON
New York City 's GPE
UHF ORG
Rockstar PERSON
Xbox ORG
PS2 ORG
Barry Vorzon 's PERSON
Bronx GPE
PlayStation 2 PRODUCT
Xbox PRODUCT
PSP ORG
@@11242 <h> PERSON
Dialect Society ORG
University of Michigan ORG
Anne Curzan PERSON
Boston GPE
the American Dialect Society 's ORG
Rina Miller PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Twittersphere ORG
Rina PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Rina PERSON
Anne PERSON
Senate ORG
Todd Akins PERSON
Rina PERSON
Anne PERSON
YOLO ORG
Drake PERSON
YOLO ORG
YOLO GPE
YOLO ORG
Pocalypse GPE
Rina PERSON
Anne PERSON
Mitt Romney PERSON
French NORP
Grant City GPE

the Viceroy 's Palace ORG
Mexico City GPE
CyArk FAC
Laser Scanning MissionCyArk ORG
Pompeii GPE
Italy GPE
Tikal GPE
Guatemala GPE
Elizabeth Lee PERSON
CyArk ORG
Ben PERSON
Barbara Kacyra PERSON
CAD ORG
the Bamiyam Buddhas WORK_OF_ART
Taliban ORG
Barbara Kacyra PERSON
> Hidden Mural Revealed < WORK_OF_ART
Historic ORG
Mexico GPE
Indians NORP
Mission Dolores ORG
CyArk PERSON
Mexico GPE
Mission LOC
Indians NORP
Andrew PERSON
Saint Joachim PERSON
Indians NORP
Mission Dolores ORG
Barbara Kacyra PERSON
Manzanar PERSON
Angor Wat PERSON
Pompeii PERSON
San Francisco < GPE
Indians NORP
Mission Dolores GPE
Andy Galvan PERSON
Museum Curator ORG
Andrew PERSON
Ohlone Indian NORP
the Mission Dolores @ @ @ @ @ @ @ @ @ FAC
San Francisco GPE
California GPE
The Digital El Camino Real ORG
the California Missions FAC
Andy Galvan 's PERSON
Ohlone Indian NORP
Mission Dolores ORG
Localore ORG
KQED ORG
Localore ORG
the Association of Independents in Radio ORG
the Corporation for Public Broadcasting ORG
MPAA Lo

Charles Darwin 's PERSON
The Expression of the Emotions in Manand Animals WORK_OF_ART
Finzi PERSON
Botox GPE
Axel Wollmer PERSON
the University of Basel ORG
Switzerland GPE
Botox GPE
the Journal of Psychiatric Research ORG
Botox GPE
Botox GPE
Botox GPE
Botox GPE
Finzi PERSON
Botox GPE
Montreal GPE
Ryerson University 's ORG
Diversity Institute ORG
the Desautels Faculty of Management at McGill ORG
DiversityLeads ORG
Social Sciences and Humanities Research Council ORG
SSHRC PERSON
Canada GPE
Agencies , Boards and Commissions ORG
Montreal GPE
Wendy Cukier PERSON
the Diversity Institute ORG
Ryerson University ORG
Daily WORK_OF_ART
Daily WORK_OF_ART
Suzanne Gagnon PERSON
Desautels PERSON
Gagnon PERSON
Elizabeth Groeneveld PERSON
the Women 's Studies ORG
McGill GPE
Montreal GPE
Groeneveld PERSON
Gagnon PERSON
Groeneveld PERSON
Cukier PERSON
DiversityLeads ORG
Aboriginal NORP
LGBT NORP
Gagnon PERSON
Canadian NORP
Vancouver GPE
Toronto GPE
@@101240 < PERSON
PA ORG
Reform ORG
The Case for Privat

America GPE
Columbian Exposition ORG
Durica PERSON
Savage PERSON
the Eden Musee GPE
Palmer Castle FAC
Durica PERSON
Bozo PERSON
Hideout ORG
Cooky NORP
Wizzo PERSON
Daily Edition WORK_OF_ART
Death : ORG
The Innkeepers WORK_OF_ART
Down Terrace WORK_OF_ART
Hobo With a Shotgun WORK_OF_ART
VHS ORG
S-VHS ORG
Sundance PRODUCT
Magnolia Pictures ORG
Ant Timpson PERSON
Drafthouse Films ' ORG
Tim League ORG
Adam Wingard PERSON
Down Terrace PERSON
Sightseers NORP
Ben Wheatley PERSON
Sundance PRODUCT
Jorge Michel Grau PERSON
Timo Tjahjanto PERSON
Safe Haven GPE
S-VHS ORG
Yoshihiro Nishimura PERSON
Tokyo GPE
Roman NORP
Nacho Vigalondo 's PERSON
Apocalypse WORK_OF_ART
Dogfight WORK_OF_ART
Marcel Sarmiento 's PERSON
Iguchi GPE
Tjahjanto GPE
Libido PERSON
Banjong Pisanthanakun PERSON
Thai NORP
Shutter ORG
Pisanthanakun ORG
Hydro-Electric Diffusion ORG
Lee Hardcastle 's PERSON
Toilet ORG
Brit NORP
Wheatley PERSON
Zetsumetsu WORK_OF_ART
Japanese NORP
Yoshihiro Nishimura PERSON
Xavier Gens ' PERSON
French

National University of Singapore ORG
Nanyang Technological University and Singapore Management University ORG
The Straits Times ORG
ST Digital ORG
@@161244 <p PERSON
Sony ORG
See The Future WORK_OF_ART
Blizzard LOC
Diablo III PRODUCT
PlayStation 3 PRODUCT
Blizzard Entertainment ORG
Diablo III GPE
Diablo III WORK_OF_ART
RPG ORG
PlayStation 3 PRODUCT
the PlayStation Network PRODUCT
Diablo ORG
PlayStation PRODUCT
Blizzard North PERSON
Diablo ORG
PlayStation PRODUCT
Blizzard GPE
Runic Games EVENT
RPG ORG
Xbox LIVE ORG
Arcade PRODUCT
RMAH ORG
Diablo ORG
PlayStation PRODUCT
Blizzard GPE
Diablo III PRODUCT
@@171240 ORG
Mobile USA ORG
Bellevue GPE
Mobile GPE
New York GPE
Mobile GPE
Mobile GPE
Seattle GPE
McCaw Cellular ORG
Mobile GPE
Bellevue GPE
Mobile GPE
Dallas GPE
MetroPCS ORG
Michele Clark PERSON
Mobile GPE
Mobile GPE
McCaw ORG
Redmond GPE
the Western U.S. LOC
Fred Devereux PERSON
West LOC
Mobile GPE
Factoria GPE
Sharpie PRODUCT
Brier Dudley PERSON
Brier Dudley PERSON
Brier Dudley PERSON


...

Norwegian NORP
Castilla GPE
Spanish NORP
Ronaldo PERSON
Ballon d'Or WORK_OF_ART
@@2981241 ORG
Cape Town - WORK_OF_ART
North Korea 's GPE
Air Koryo ORG
Skytrax ORG
UK GPE
Daily Mail ORG
Air Koryo ORG
EU ORG
Air Koryo 's ORG
East LOC
North Korea GPE
Petri Van Zyl ORG
North Korea 's GPE
Kosmonooit - Cockpit ORG
Blackwolf ORG
Congo GPE
Nigeria GPE
Comments Policy WORK_OF_ART
Comment PERSON
@@2981242 <h> Rental PERSON
Johannesburg GPE
Hellopeter PRODUCT
Dave Pullen PERSON
VW ORG
Polo Vivo 1.4 PRODUCT
First Car Rental ORG
Tambo ORG
Pullen PERSON
R167 PERSON
First Car Rental 's ORG
Tambo PERSON
Pomona GPE
Pullen PERSON
Pullen PERSON
Pullen PERSON
Vivo GPE
Pullen PERSON
Pullen PERSON
Pump Record Sheet ORG
Melissa Storey PERSON
Strategy , Development &amp ORG
First Car Rental ORG
First Car Rental ORG
Vivo GPE
Polo Vivo FAC
Comment Guidelines PERSON
Flag PRODUCT
Verified PERSON
Independent Media ORG
Kickstarter Campaign for Exploding Kittens ORG
Cassidee Moser < PERSON
Kickstarter ORG
Russian NO

Greek NORP
Microsoft ORG
SQL ORG
Hekaton LOC
MSDN ORG
SQL Server ORG
SQL Server LOC
SQL ORG
SQL ORG
RAM PRODUCT
SQL ORG
SQL Server ORG
SQL Server LOC
the New York Times ORG
San Francisco GPE
Twitter ORG
Dick Costolo PERSON
Twitter ORG
Costolo ORG
Twitter ORG
Costolo ORG
Facebook ORG
Twitter ORG
Facebook ORG
Costolo ORG
Twitter ORG
Correction ORG
Costolo ORG
Costolo PERSON
our Privacy Policy WORK_OF_ART
Facebook ORG
Facebook ORG
@@3041240 GPE
Baha'i ORG
Australian NORP
Shameem <p> PERSON
Grammy WORK_OF_ART
Perth GPE
Baha'i ORG
Shameem PERSON
Geoff Wood PERSON
MySpace ORG
Shameem PERSON
Grammy WORK_OF_ART
MySpace ORG
James Bryan PERSON
UK GPE
Baha'i ORG
Shameem PERSON
British NORP
Perth GPE
Shameem Taheri-Lee PERSON
James Bryan PERSON
Nelly Furtado PERSON
Olly Murs PERSON
Shameem PERSON
The Second City GPE
Neo PERSON
Jill Scott PERSON
Erykah Badu PERSON
Alicia Keys PERSON
Alicia Keys PERSON
Shameen ORG
Alicia Keys PERSON
Baha'i ORG
Baha'i ORG
Baha'i ORG
Baha'i ORG
Shameem PERSON
Baha'i O

Windsor PERSON
Dan Tanel PERSON
Badgers ORG
Badgers ORG
Badgers ORG
Julian Luciani PERSON
Badgers ORG
Chad Sheply PERSON
Lancer PRODUCT
Volpe PERSON
Badgers ORG
Spencer Turcotte PERSON
Brock PERSON
Lancers PRODUCT
Luciani ORG
Kyle Hope PERSON
Volpe PERSON
Badgers ORG
> The Badgers WORK_OF_ART
Windsor ORG
OUA ORG
Badgers ORG
Nystrom @ @ @ PERSON
Adrian Volpe PERSON
Jordan Gignac PERSON
Kaine PERSON
Anthony Geldart PERSON
Mike McGurk PERSON
Brock Badgers ORG
Windsor Lancers ORG
Henry Holtmann PERSON
Henry Holtmann PERSON
Dairy Farmers ORG
Manitoba GPE
Holtmann PERSON
Holtmann GPE
Holtmann PERSON
Rosser Holsteins PERSON
Holtmann PERSON
Holtmann GPE
WiFi ORG
Holtmann GPE
Holtmann PERSON
Systems ORG
Epple PERSON
Epple PERSON
Holtmann PERSON
Farmers ORG
Holtmann GPE
Holtmann GPE
Holtmann PERSON
Holtmann PERSON
Holtmann PERSON
Rosser Holsteins ORG
Holtmann PERSON
Holtmann PERSON
Holtmann PERSON
Holtmann PERSON
Navy ORG
HMS ORG
Elizabeth PERSON
Rosyth Dock PERSON
Fife GPE
the River Forth FAC
t

Servo ORG
Earth LOC
Bloom WORK_OF_ART
C&amp;C ORG
Multiplayer ORG
@@3091244 < PERSON
Meryl Streep PERSON
Australia GPE
Fred Schepisi PERSON
Evil Angels ORG
Lindy Chamberlain PERSON
Australian NORP
Australian NORP
Australian NORP
Australia GPE
Meryl Streep PERSON
Lindy Chamberlain PERSON
Evil Angels ORG
American NORP
Meryl Streep PERSON
Americans NORP
the Temporary Work ( Entertainment ORG
Subclass GPE
the Media , Entertainment and Arts Alliance ORG
Australian NORP
Australian NORP
Independent ORG
Nick Xenophon PERSON
Texas GPE
Australian NORP
Susie Porter PERSON
Puberty Blues ORG
Dan Wyllie PERSON
Brenna Harding PERSON
Australian NORP
Australian NORP
Sarah Snook PERSON
Predestination WORK_OF_ART
American NORP
Sarah Snook PERSON
Predestination WORK_OF_ART
American NORP
Ethan Hawke PERSON
US GPE
American NORP
American NORP
Australian NORP
Roy Billing PERSON
MEAA ORG
Zoe Angus PERSON
MEAA ORG
SPA ORG
Matthew @ @ PERSON
the Immigration Department ORG
Foxtel ORG
Australian NORP
MEAA ORG
Aust

KeyboardInterrupt: 

In [16]:
keyString = ""
for key in keywords:
    keyString += key['text'].title()
keytokens = nlp(keyString)

with open('refined_named_entities.txt', 'r') as file:
    tokens = nlp(file.read())

In [31]:
dup = set()
with open("Questions3.txt", "w") as file:
    for key in keytokens:
        for token in tokens:
            tokenSimilarity = token.similarity(key)
            if tokenSimilarity > 0.6 and tokenSimilarity != 1 and token.text not in dup and len(token.text) > 2:
                file.write("What is the difference between %s and %s?\n" % (token.text, key.text))
                dup.add(token.text)

In [12]:
# lines_seen = set() # holds lines already seen
# outfile = open("refined_named_entities.txt", "w")
# for line in open("named_entities.txt", "r"):
#     if line not in lines_seen: # not a duplicate
#         outfile.write(line)
#         lines_seen.add(line)
# outfile.close()